In [3]:
import requests
import json

def get_transaction_details(intent_hash):
    """
    Fetch transaction details from the Radix network.
    
    Args:
        intent_hash (str): The transaction intent hash to query
        
    Returns:
        requests.Response: The response object from the API request
    """
    # Define the endpoint URL
    url = 'https://mainnet.radixdlt.com/transaction/committed-details'

    # Define the headers
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }

    # Define the payload data
    payload = {
        "intent_hash": intent_hash,
        "opt_ins": {
            "raw_hex": True,
            "receipt_state_changes": True,
            "receipt_fee_summary": True,
            "manifest_instructions": True,
            "receipt_fee_source": True,
            "receipt_fee_destination": True,
            "receipt_costing_parameters": True,
            "receipt_events": True,
            "receipt_output": True,
            "affected_global_entities": True,
            "balance_changes": True
        }
    }

    # Make the POST request
    return requests.post(url, headers=headers, data=json.dumps(payload))


# Example usage
# response = get_transaction_details("txid_rdx1aqqwq3e95k4ktr5yt2s7sgtnsnf64uh87gase8k74we3fe5fyx3q7tzvls")
# response = get_transaction_details("txid_rdx1arc8vukn4j5lg0vfrjdjxl3rf0e7uh8vc22t4wjhqq3s0pz6xnvqq6f88p")
# response = get_transaction_details("txid_rdx1nvh6xsj03ykm0r03ppny4qj9xtjtl9vaf4hw5n77ammg0sedr4zqnz43xf")
# response = get_transaction_details("txid_rdx16jethsv50jvj7nalsqqa2zwvzhps04avx9k5gal3qukgp3a8sjsqjq077a") # Adam's example txn
txn_intent_hash = "txid_rdx1arc8vukn4j5lg0vfrjdjxl3rf0e7uh8vc22t4wjhqq3s0pz6xnvqq6f88p"
txn_details = get_transaction_details(txn_intent_hash)

txn_details.json()
state_version = txn_details.json()['transaction']['state_version']   
state_version


286232755

In [4]:
# Define a function to get transaction stream data
def get_transaction_stream(state_version):
    """
    Get transaction stream data from the Radix API.
    
    Args:
        state_version (int): The state version to query
        
    Returns:
        requests.Response: The response object from the API request
    """
    # Define the endpoint URL
    url = 'https://mainnet.radixdlt.com/stream/transactions'
    
    # Define the headers
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    
    # Define the payload data
    payload = {
        "at_ledger_state": {
            "state_version": state_version
        },
        "opt_ins": {
            "detailed_events": True
        },
        "transaction_status_filter": "Success"
        
    }
    
    # Make the POST request
    return requests.post(url, headers=headers, data=json.dumps(payload))

# Call the function with the state_version we got earlier
response = get_transaction_stream(state_version)
parsed_response = response.json()
transaction_data = [x for x in parsed_response.get('items') if x.get('intent_hash') == txn_intent_hash]
transaction_data






[{'transaction_status': 'CommittedSuccess',
  'state_version': 286232755,
  'epoch': 202945,
  'round': 185,
  'round_timestamp': '2025-05-13T08:39:42.802Z',
  'payload_hash': 'notarizedtransaction_rdx1083vywn2ch8du8agx5u3dcqk8zm3q56wtsa7eqp0rerxrhlr2neqx0cvep',
  'intent_hash': 'txid_rdx1arc8vukn4j5lg0vfrjdjxl3rf0e7uh8vc22t4wjhqq3s0pz6xnvqq6f88p',
  'fee_paid': '4.391931913313333333',
  'confirmed_at': '2025-05-13T08:39:42.802Z',
  'receipt': {'status': 'CommittedSuccess',
   'detailed_events': [{'identifier': {'package': 'package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx000538436477xxxxxxxxxresrce',
      'blueprint': 'FungibleVault',
      'event': 'LockFeeEvent'},
     'payload_type_definition': {'local_type_id': 22,
      'entity': 'package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx000538436477xxxxxxxxxresrce',
      'schema_hash_hex': '462a3fea283117aab2b01c297812bdc0fa9060b29eb5e68b847f361bc1201933',
      'kind': 'SchemaLocal'},
     'emitter': {'method_emitter': {'entity': 'internal_vault_rdx1tq6e

In [5]:
import pandas as pd


events = transaction_data[0]['receipt']['detailed_events']
# Extract data from events
events

# Initialize an empty list to store event data
event_data = []

# Process each event and extract relevant information
for event in events:
    # Initialize a dictionary for this event
    event_dict = {}
    
    # Extract identifier information
    if 'identifier' in event:
        event_dict['name'] = event['identifier'].get('event', '')
        event_dict['blueprint'] = event['identifier'].get('blueprint', '')
        event_dict['package'] = event['identifier'].get('package', '')
    
    # Extract emitter information
    if 'emitter' in event:
        event_dict['emitter_type'] = event['emitter'].get('type', '')
        
        # Handle different emitter structures
        if 'method_emitter' in event['emitter'] and 'entity' in event['emitter']['method_emitter']:
            entity = event['emitter']['method_emitter']['entity']
            if isinstance(entity, str):
                event_dict['entity_address'] = entity
            elif isinstance(entity, dict):
                event_dict['entity_address'] = entity.get('entity_address', '')
        
        if 'global_emitter' in event['emitter']:
            event_dict['global_emitter'] = event['emitter']['global_emitter']
    
    # Extract payload information
    if 'payload' in event and 'programmatic_json' in event['payload']:
        payload = event['payload']['programmatic_json']
        event_dict['data_kind'] = payload.get('kind', '')
        
        # Extract fields from payload based on structure
        if payload.get('kind') == 'Tuple' and 'fields' in payload:
            for field in payload['fields']:
                if 'field_name' in field and 'value' in field:
                    event_dict[field['field_name']] = field['value']
        
        # Handle Enum type payloads
        elif payload.get('kind') == 'Enum' and 'fields' in payload:
            event_dict['variant_name'] = payload.get('variant_name', '')
            
            for field in payload['fields']:
                if isinstance(field, dict):
                    if 'field_name' in field and 'value' in field:
                        event_dict[field['field_name']] = field['value']
                    elif 'type_name' in field and 'value' in field:
                        event_dict[field['type_name']] = field['value']
    
    # Add the event dictionary to our list
    event_data.append(event_dict)

# Create a pandas DataFrame from the event data
events_df = pd.DataFrame(event_data)

# Display the DataFrame
events_df




,name,blueprint,package,emitter_type,entity_address,global_emitter,data_kind,amount,variant_name,ResourceAddress,...,global_x_fee_lp,global_y_fee_lp,token_address,amount_change_x,amount_change_y,price_after,amount_after_x,amount_after_y,base_amount,quote_amount
0,LockFeeEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tq6emj939c4qejwtd24jvfvhwq7...,account_rdx16ykn77yw3mnmjtc2q3dx5qnxml7gap33fp...,Tuple,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WithdrawEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tz5mypwz7vzv3cxqwann0zkawey...,account_rdx16ykn77yw3mnmjtc2q3dx5qnxml7gap33fp...,Tuple,0.02219,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WithdrawEvent,Account,package_rdx1pkgxxxxxxxxxaccntxxxxxxxxxx0009296...,EntityMethod,account_rdx16ykn77yw3mnmjtc2q3dx5qnxml7gap33fp...,account_rdx16ykn77yw3mnmjtc2q3dx5qnxml7gap33fp...,Enum,NaN,Fungible,resource_rdx1t5pyvlaas0ljxy0wytm5gvyamyv896m69...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BurnFungibleResourceEvent,FungibleResourceManager,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,resource_rdx1t5pyvlaas0ljxy0wytm5gvyamyv896m69...,resource_rdx1t5pyvlaas0ljxy0wytm5gvyamyv896m69...,Tuple,0.02219,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BurnEvent,FeeVaults,package_rdx1p42zvel2uhtamrpwtuee9e80hhwm3hlcwa...,EntityMethod,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,Tuple,0.02219,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DepositEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tzux9934ev23sy99plx5fv0e5r6...,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,Tuple,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ReserveDepositEvent,FeeVaults,package_rdx1p42zvel2uhtamrpwtuee9e80hhwm3hlcwa...,EntityMethod,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,Tuple,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,WithdrawEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tqt3q4aj92rgrqguvt3jahlpwuz...,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,Tuple,48630.228887498579994582,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SwapEvent,FeeVaults,package_rdx1p42zvel2uhtamrpwtuee9e80hhwm3hlcwa...,EntityMethod,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,Tuple,48630.228887498579994582,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DepositEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1trs0xmx638vcn72e8uyc0ct020t...,component_rdx1czre754pv64wd74v4mn80769jy5l3p63...,Tuple,48.630228887498579994,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:

# Filter out SwapEvent from the dataframe
swap_events = events_df[events_df['name'].isin(['SwapEvent', 'SwapVaultDepositEvent'])]

# Select and display the requested columns
if not swap_events.empty:
    swap_details = swap_events[[
        'name',
        'entity_address',
        'amount',
        'input_address',
        'input_amount',
        'input_fee_lp',
        'input_fee_protocol',
        'output_address',
        'output_amount'
    ]].reset_index(drop=True)
    
    print("Swap Event Details:")
    display(swap_details)
else:
    print("No SwapEvent found in the data.")


Swap Event Details:


,name,entity_address,amount,input_address,input_amount,input_fee_lp,input_fee_protocol,output_address,output_amount
0,SwapEvent,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,48630.228887498579994582,NaN,NaN,NaN,NaN,NaN,NaN
1,SwapEvent,component_rdx1czre754pv64wd74v4mn80769jy5l3p63...,NaN,resource_rdx1t466mhd2l2jmmzxr8cg3mkwjqhs7zmjgt...,48143.926598623594194636,437.672059987487219952,48.630228887498579994,resource_rdx1tknxxxxxxxxxradxrdxxxxxxxxx009923...,2999.14369937397784692
2,SwapEvent,component_rdx1crw3x8qef9kskk524cz9euhwpaqw7l22...,NaN,resource_rdx1tknxxxxxxxxxradxrdxxxxxxxxx009923...,74.949388801101576973,0.224848166403304731,NaN,resource_rdx1t5ywq4c6nd2lxkemkv4uzt8v7x7smjcgu...,57.687305122145272265
3,SwapVaultDepositEvent,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,0.51713016063775376,NaN,NaN,NaN,NaN,NaN,NaN
4,SwapEvent,component_rdx1czgaazn4wqf40kav57t8tu6kwv2a5sfm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SwapEvent,component_rdx1crvsxzkyh0609z4jj7vjzwrnede37676...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SwapEvent,component_rdx1crmhyn4m3u3pxpx74lafpz5yyjtlyupx...,NaN,resource_rdx1tknxxxxxxxxxradxrdxxxxxxxxx009923...,339.405360944584458593,1.018216082833753376,NaN,resource_rdx1t5pyvlaas0ljxy0wytm5gvyamyv896m69...,0.002522879075216956
7,SwapVaultDepositEvent,component_rdx1crmhkatyjrw0070nsusdm4adwr5s3eay...,0.258306524250733782,NaN,NaN,NaN,NaN,NaN,NaN
8,SwapEvent,component_rdx1cz839fe20qs607t2el79wct6wmg47s0v...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
